In [3]:
# problem statement 

In [4]:
# importing the libraries 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import sqlite3 
%matplotlib inline

In [5]:
import sqlite3
conn = sqlite3.connect("classification.db")
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(tables)

[('fraud_detection',), ('Heart_disease',), ('Metro_Train_Maintenance',), ('forest_cov_type',), ('fake_news_classification',), ('drug_reviews_sentiment_analysis',), ('Telecom_microservices_loan',), ('malware_detection',), ('fault_detection_manufacturing',)]


In [6]:
#connect to the SQlite database file
conn = sqlite3.connect("classification.db")


#load the table from the database
df=pd.read_sql("select* FROM fraud_detection",conn)

# close the connection
conn.close()




In [7]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [8]:
df.shape

(6362620, 11)

In [9]:
df.isnull().sum()

step                   0
type              203603
amount                 0
nameOrig          203603
oldbalanceOrg          0
newbalanceOrig    267230
nameDest               0
oldbalanceDest    203603
newbalanceDest         0
isFraud                0
isFlaggedFraud         0
dtype: int64

In [11]:
df.columns

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [15]:
df.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.095390e+06,6.159017e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,8.338831e+05,8.550005e+05,1.101147e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.888243e+06,2.923541e+06,3.408166e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327682e+05,2.146614e+05,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087215e+05,1.073152e+05,1.442704e+05,9.431247e+05,1.111909e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


checking the unique values present in the column ignore numeric column 


In [20]:
#checking the unique value in the type colummns 
df['type'].unique()

array(['PAYMENT', 'TRANSFER', 'CASH_OUT', 'DEBIT', None, 'CASH_IN'],
      dtype=object)

**intepreatation** no all are the unique values 

In [21]:
#checking the duplicates in the dataset 
df.duplicated().sum()

np.int64(0)

**Interpretation** :-there is no duplicate values present in the DF

#### checking the missing value 

In [17]:
total_row= len(df)
for col in df.columns:
    missing_percentage=(df[col].isnull().sum() / total_row)*100
    print(f'{col}:{missing_percentage:.2f}%')


step:0.00%
type:3.20%
amount:0.00%
nameOrig:3.20%
oldbalanceOrg:0.00%
newbalanceOrig:4.20%
nameDest:0.00%
oldbalanceDest:3.20%
newbalanceDest:0.00%
isFraud:0.00%
isFlaggedFraud:0.00%


**Interpretation:-** missing value percentage is less than 5% so we can delete the row for the better results 